# Converter for the Financial Filings at the Sec

In [3]:
from zipfile import ZipFile
import pandas as pd

dirname = "2022q4"
with ZipFile('importFiles/'+dirname+'.zip') as myzip:
    with myzip.open('num.txt') as myfile1:
        dfNum = pd.read_table(myfile1,delimiter="\t", low_memory=False)
    with myzip.open('pre.txt') as myfile2:
        dfPre = pd.read_table(myfile2,delimiter="\t")
    with myzip.open('sub.txt') as myfile3:
        dfSub = pd.read_table(myfile3,delimiter="\t", low_memory=False)
    with myzip.open('tag.txt') as myfile4:
        dfTag = pd.read_table(myfile4,delimiter="\t")

dfSym = pd.read_table('importFiles/ticker.txt', delimiter="\t", header=None, names=['symbol','cik'])        

print(dfNum.size)
print(dfPre.size)
print(dfSub.size)
print(dfTag.size)
print(dfSym.size)

35433920
8063710
262080
862002
24168


In [4]:
from tabulate import tabulate

print(tabulate(dfNum.head(100), headers='keys', tablefmt='psql'))
print(dfNum.dtypes)

+----+----------------------+----------------------------------------------------------------------------------------------------------------+----------------------+----------+--------+--------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------+-------------------+------------+
|    | adsh                 | tag                                                                                                            | version              |    ddate |   qtrs | uom    | segments                                                                                                                                                                                                                 | coreg                           |             value |   footnote |
|----+----------------------+---------

In [5]:
print(tabulate(dfPre.head(100), headers='keys', tablefmt='psql'))
print(dfNum.dtypes)

+----+----------------------+----------+--------+--------+---------+---------+----------------------------------------------------------------------------------------------------------------+----------------------+----------------------------------------------------------------------------------------------------------------------+------------+
|    | adsh                 |   report |   line | stmt   |   inpth | rfile   | tag                                                                                                            | version              | plabel                                                                                                               |   negating |
|----+----------------------+----------+--------+--------+---------+---------+----------------------------------------------------------------------------------------------------------------+----------------------+--------------------------------------------------------------------------------------------

In [6]:
print(tabulate(dfSub.head(100), headers='keys', tablefmt='psql'))
print(dfNum.dtypes)

+----+----------------------+--------+--------------------------------------+-------+-------------+----------+-----------------+------------+-----------------------------------------+----------------------------------------+----------------+-------------+----------+-----------------+------------+-------------------------------------+----------------------------------------+--------------+-----------+-----------+-------------------------------------------+---------------+-------+--------+-------+--------+----------+------+------+----------+-----------------------+-----------+----------+---------------------------+---------+----------------------------------------------+
|    | adsh                 |    cik | name                                 |   sic | countryba   | stprba   | cityba          | zipba      | bas1                                    | bas2                                   | baph           | countryma   | stprma   | cityma          | zipma      | mas1                    

In [7]:
print(tabulate(dfTag.head(100), headers='keys', tablefmt='psql'))
print(dfNum.dtypes)

+----+-------------------------------------------------------------------------------------------------------------------------+--------------+----------+------------+------------+--------+--------+---------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|    | tag                        

In [8]:
print(tabulate(dfSym.head(10), headers='keys', tablefmt='psql'))
print(dfNum.dtypes)

+----+----------+---------+
|    | symbol   |     cik |
|----+----------+---------|
|  0 | aapl     |  320193 |
|  1 | msft     |  789019 |
|  2 | brk-b    | 1067983 |
|  3 | unh      |  731766 |
|  4 | jnj      |  200406 |
|  5 | v        | 1403161 |
|  6 | tsm      | 1046179 |
|  7 | xom      |   34088 |
|  8 | wmt      |  104169 |
|  9 | spy      |  884394 |
+----+----------+---------+
adsh         object
tag          object
version      object
ddate         int64
qtrs          int64
uom          object
segments     object
coreg        object
value       float64
footnote     object
dtype: object


In [9]:
print(dfNum.iloc[0])
print(dfNum.columns.values)

adsh                0000046080-22-000123
tag         AccountsReceivableNetCurrent
version                     us-gaap/2022
ddate                           20220930
qtrs                                   0
uom                                  USD
segments                             NaN
coreg                                NaN
value                       1188800000.0
footnote                             NaN
Name: 0, dtype: object
['adsh' 'tag' 'version' 'ddate' 'qtrs' 'uom' 'segments' 'coreg' 'value'
 'footnote']


In [10]:
from datetime import date, datetime, timedelta
from marshmallow import Schema, fields

class FinancialElementImportDto:
    label = ""          #Tag.doc
    concept = ""        #Num.tag
    info = ""           #Pre.plabel
    unit = ""           #Num.uom
    value = 0.0         #Num.value

class FinancialsDataDto:
    bs = []  #FinancialElementImportDto[] mapping key from Pre.stmt
    cf = []  #FinancialElementImportDto[] mapping key from Pre.stmt
    ic = []  #FinancialElementImportDto[] mapping key from Pre.stmt
    

class SymbolFinancialsDto:
    startDate = date.today()   #Sub.period 
    endDate = date.today()     #Sub.period + Sub.fp
    year = 0                   #Sub.fy
    quarter = ""               #Sub.fp
    symbol = ""                #Sub.cik -> Sym.cik -> Sym.symbol
    name = ""                  #Sub.name
    country = ""               #Sub.countryma
    city = ""                  #Sub.cityma
    data = FinancialsDataDto()
    
class FinancialElementImportSchema(Schema):
    label = fields.String()
    concept = fields.String()
    info = fields.String()
    unit = fields.String()
    value = fields.Int()
    
class FinancialsDataSchema(Schema):
    bs = fields.List(fields.Nested(FinancialElementImportSchema()))
    cf = fields.List(fields.Nested(FinancialElementImportSchema()))
    ic = fields.List(fields.Nested(FinancialElementImportSchema()))
    
class SymbolFinancialsSchema(Schema):
    startDate = fields.DateTime()
    endDate = fields.DateTime()
    year = fields.Int()
    quarter = fields.String()
    symbol = fields.String()
    name = fields.String()
    country = fields.String()
    city = fields.String()
    data = fields.Nested(FinancialsDataSchema)

In [11]:
pip install marshmallow


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: /opt/anaconda3/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [12]:
import os

#isExist = os.path.exists("exportFiles/"+dirname)
isExist = 1 == 2
if isExist:
    for filename in os.listdir("exportFiles/"+dirname):
        with open(os.path.join("exportFiles/"+dirname, filename), 'r') as file:
            jsonStr = file.read()
            file.close()
        jsonStr = jsonStr.replace("'", '"').replace(", ",",").replace('\\r', '').replace('\\n', ' ')
        jsonStr = ''.join(jsonStr.splitlines())
        with open(os.path.join("exportFiles/"+dirname, filename), 'w') as file:    
            file.write(jsonStr)
            file.close()
    

# Create a single SymbolFinancialsDto

In [14]:
#json_string = json.dumps([ob.__dict__ for ob in list_name])
#rslt_df = dataframe[dataframe['Percentage'] > 80]
import numpy as np
from pprint import pprint
import os
from dateutil.relativedelta import relativedelta
import json

def npInt_to_str(var):
    return str(list(np.reshape(np.asarray(var), (1, np.size(var)))[0]))[1:-1]

def formatDateNpNum(var):
    dateStr = npInt_to_str(var)
    return dateStr[0:4]+"-"+dateStr[4:6]+"-"+dateStr[6:8]

fileStartTime = datetime.now()
for subId in range(len(dfSub)):
    startTime = datetime.now()
    submitted = dfSub.iloc[subId]
    sfDto = SymbolFinancialsDto()
    sfDto.data = FinancialsDataDto()
    sfDto.data.bs = []
    sfDto.data.cf = []
    sfDto.data.ic = []
    try:
        periodStartDate = date.fromisoformat(formatDateNpNum(submitted["period"]))
    except ValueError:
        print("File exportFiles/"+dirname+"/"+submitted["adsh"]+".json has Period: "+str(submitted["period"]))
        continue       
    sfDto.startDate = periodStartDate
    sfDto.endDate = date.today() #default value
    if submitted["fy"] == np.nan or np.isnan(submitted["fy"]):
        sfDto.year = 0
    else: 
        sfDto.year = submitted["fy"].astype(int)        
    sfDto.quarter = str(submitted["fp"]).strip().upper()
    
    if sfDto.quarter == "FY" or sfDto.quarter == "CY":
        sfDto.endDate = periodStartDate + relativedelta(months=+12, days=-1)
    elif sfDto.quarter == "H1" or sfDto.quarter == "H2":
        sfDto.endDate = periodStartDate + relativedelta(months=+6, days=-1)
    elif sfDto.quarter == "T1" or sfDto.quarter == "T2" or sfDto.quarter == "T3":
        sfDto.endDate = periodStartDate + relativedelta(months=+4, days=-1)
    elif sfDto.quarter == "Q1" or sfDto.quarter == "Q2" or sfDto.quarter == "Q3" or sfDto.quarter == "Q4":
        sfDto.endDate = periodStartDate + relativedelta(months=+3, days=-1)
    else:
        continue
        
    #symStr = dfSym[dfSym["cik"]==submitted.cik].symbol.str.upper().astype("string")
    #print(symStr)
    val = dfSym[dfSym["cik"]==submitted["cik"]]
    if len(val) > 0:
        sfDto.symbol = val["symbol"].to_string(index = False).split("\n")[0].split("\\n")[0].strip().split(" ")[0].strip().upper()
        if len(sfDto.symbol) > 19 or len(sfDto.symbol) < 1:
            print("File exportFiles/"+dirname+"/"+submitted["adsh"]+".json has Symbol "+sfDto.symbol)
    else:
        print("File exportFiles/"+dirname+"/"+submitted["adsh"]+".json has Symbol "+val["symbol"].to_string(index = False))
        continue
        
    sfDto.name = submitted["name"]
    sfDto.country = submitted["countryma"]
    sfDto.city = submitted["cityma"]

    sdDto = FinancialsDataDto()
    #print(filteredDfPre)
    dfNum['value'] = pd.to_numeric(dfNum['value'], errors='coerce')
    dfNum = dfNum.dropna(subset=['value'])
    dfNum['value'] = dfNum['value'].astype(int)
    #filteredDfNum = dfNum[dfNum['adsh'] == submitted['adsh']]       
    filteredDfNum = dfNum.loc[dfNum['adsh'].values == submitted['adsh']]
    filteredDfNum.reset_index()

    for myId in range(len(filteredDfNum)): 
        #print(myId)
        #print(filteredDfNum.iloc[myId])
        myNum = filteredDfNum.iloc[myId]
        myDto = FinancialElementImportDto()
        myTag = dfTag[dfTag["tag"] == myNum['tag']]
        myDto.label = myTag["doc"].to_string(index = False)
        myDto.concept = myNum["tag"]
        #myPre = dfPre[(dfPre['adsh'] == submitted["adsh"]) & (dfPre['tag'] == myNum['tag'])]
        myPre = dfPre.loc[np.where((dfPre['adsh'].values == submitted["adsh"]) & (dfPre['tag'].values == myNum['tag']))]
        myDto.info = myPre["plabel"].to_string(index = False).replace('"',"'")
        myDto.unit = myNum["uom"]
        #print(myNum["value"])        
        myDto.value = myNum["value"]
        #print(myDto.__dict__)
        #FinancialElementImportSchema().dump(myDto)
        if myPre['stmt'].to_string(index = False).strip() == 'BS':
            sfDto.data.bs.append(myDto)
        elif myPre['stmt'].to_string(index = False).strip() == 'CF':
            sfDto.data.cf.append(myDto)
        elif myPre['stmt'].to_string(index = False).strip() == 'IC':
            sfDto.data.ic.append(myDto)

    #print(len(filteredDfNum))
    #print(submitted)
    #print(sfDto.__dict__)    
    #json.dumps(sfDto.__dict__)    
    marshmellowStart = datetime.now()
    result = SymbolFinancialsSchema().dump(sfDto)
    result = str(json.dumps(result))
    #result = result.replace('"','öäü').replace("'", '"').replace("öäü","'").replace(", ",",")
    result = result.replace('\\r', '').replace('\\n', ' ')
    
    print("FilterDfNum size: "+str(len(filteredDfNum))+" Dtos size: "+str((len(sfDto.data.bs)+len(sfDto.data.cf)+len(sfDto.data.ic)))+" Bs: "+str(len(sfDto.data.bs))+" Cf: "+str(len(sfDto.data.cf))+" Ic: "+str(len(sfDto.data.ic))+" Json size: "+str(int(len(str(result)) / 1024))+"kb.")
    
    #print("Json size "+str(int(len(str(result)) / 1024))+"kb. Time: "+str((datetime.now() - marshmellowStart) / timedelta(milliseconds=1))+"ms.")
    
    #print(result)
    #pprint(result)    
    isExist = os.path.exists("exportFiles/"+dirname)
    if not isExist:
        os.makedirs("exportFiles/"+dirname)
    json_file = open("exportFiles/"+dirname+"/"+submitted["adsh"]+".json","w")
    json_file.write(result)
    json_file.close()
    endTime = datetime.now()
    print("file "+str(subId+1)+" of "+str(len(dfSub)+1)+" exportFiles/"+dirname+"/"+submitted["adsh"]+".json stored in "+str((endTime - startTime) / timedelta(milliseconds=1))+"ms.")
    
fileEndTime = datetime.now()
print("Time "+str((fileEndTime - fileStartTime) / timedelta(seconds=1))+"sec for "+str(len(dfSub)+1)+" files.")

FilterDfNum size: 485 Dtos size: 155 Bs: 100 Cf: 55 Ic: 0 Json size: 37kb.
file 1 of 7281 exportFiles/2022q4/0000002178-22-000089.json stored in 5663.978ms.
FilterDfNum size: 401 Dtos size: 163 Bs: 96 Cf: 67 Ic: 0 Json size: 39kb.
file 2 of 7281 exportFiles/2022q4/0000002488-22-000170.json stored in 4667.638ms.
FilterDfNum size: 772 Dtos size: 287 Bs: 140 Cf: 147 Ic: 0 Json size: 71kb.
file 3 of 7281 exportFiles/2022q4/0000002969-22-000054.json stored in 8924.114ms.
FilterDfNum size: 274 Dtos size: 116 Bs: 72 Cf: 44 Ic: 0 Json size: 27kb.
file 4 of 7281 exportFiles/2022q4/0000003499-22-000027.json stored in 3250.736ms.
FilterDfNum size: 809 Dtos size: 264 Bs: 179 Cf: 85 Ic: 0 Json size: 86kb.
file 5 of 7281 exportFiles/2022q4/0000003570-22-000106.json stored in 9371.11ms.
FilterDfNum size: 403 Dtos size: 192 Bs: 86 Cf: 106 Ic: 0 Json size: 46kb.
file 6 of 7281 exportFiles/2022q4/0000004127-22-000038.json stored in 4803.292ms.
FilterDfNum size: 598 Dtos size: 187 Bs: 85 Cf: 102 Ic: 0 Js

KeyboardInterrupt: 